In [1]:
####################################################################
# bertwordpiece 방식으로 토큰 사전 생성 
#
# 1) mecab 로 한번 말뭉치 분석
# 2) 분석된 말뭉치를 가지고 wordpiece 토큰화(훈련)
# 3) special token 추가
####################################################################

#변수들 지정 
#
# input : 말뭉치 경로
# mecab_output : 입력된 말뭉치를 mecab 으로 형태소 분석한 output 파일 
# mecab_generation = True 이면, 형태소 앞에 ## 붙임, False면 안붙임

input = '../../data11/ai_hub/tl1/tl1-1줄.txt'
#mecab_output = '../../data11/my_corpus/re-kowiki-202206-mecab.txt' # ["wiki_20190620_small.txt", "my_data/nsmc_subword.txt"], 여러개 파일도 추가할수 있음
mecab_output = '../../data11/ai_hub/tl1/tl1-1줄-mecab.txt' # ["wiki_20190620_small.txt", "my_data/nsmc_subword.txt"], 여러개 파일도 추가할수 있음
mecab_generation = False

vocab_size = 30000   # 출력 vocab 크기
min_frequency = 200  # 빈도수

# 출력 경로 
vocab_folder = '../../data11/ai_hub/vocab/' #출력 폴더(*반드시 우선 생성해야함)
vocab_output = 'tl1-1줄-mecab-30000'  #출력 파일명
vocab_special_output = 'tl1-1줄-mecab-30000-speical'   # tokenizer 생성할때 파일명

# 출력폴더 없으면 생성
import os
os.makedirs(vocab_folder, exist_ok=True)

#seed 설정 해도 vocab 은 다르게 생성됨
#import sys
#sys.path.append("..")
#from myutils import seed_everything
#seed_everything(111)

#os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
## yield 를 이용하여 generator 함수 정의  
## => 10만 단위로 쪼갬
def get_generator_corpus(data, max_len: int=100000):
    
    dataset = data
    #dataset = list(set(data)) # ****중복 문장 제거(*순서 유지 안함)
    
    for start_idx in range(0, len(dataset), max_len):
        samples = dataset[start_idx : start_idx + max_len]
        yield samples

In [ ]:
##### Mecab 를 이용하여, 말뭉치 형태소 분석 
import konlpy
from konlpy.tag import Mecab
from tqdm.notebook import tqdm

# 말뭉치 불러오기
with open(input, 'r', encoding='utf-8') as f:
    data = f.read().split('\n')
    
print(data[:3])
print(len(data))

# mecab 출력 파일 열기
f1 = open(mecab_output, 'w', encoding='utf-8')

# 10만 단위에 generator 정의
gen_corpus = get_generator_corpus(data)

mecab = Mecab()

# True: '어릴때' -> '어릴', '##'때' 로 일반화 시킴
#sentence_count = 0
if mecab_generation == True:
  total_morph=[]
  for sentences in tqdm(gen_corpus):
    for sentence in sentences:
        morph_sentence=[]
        count=0
        for token_mecab in mecab.morphs(sentence):
          token_mecab_save = token_mecab
          if count > 0:
            token_mecab_save = "##" + token_mecab_save
            morph_sentence.append(token_mecab_save)
          else:
            morph_sentence.append(token_mecab_save)
            count += 1

        f1.write(' '.join(morph_sentence)+'\n')
    
    #if sentence_count % 1000000 == 0:
    #    print('sentence_count:{}'.format(sentence_count))
    
    #sentence_count += 1
        
# False: '어릴때' -> '어릴', '때' 로 일반화 시킴
else:
  total_morph=[]
  for sentences in tqdm(gen_corpus):
    for sentence in sentences:
        morph_sentence = mecab.morphs(sentence)
        f1.write(' '.join(morph_sentence)+'\n')
          
        #total_morph.append(morph_sentence)
   
    #if sentence_count % 1000000 == 0:
    #   print('sentence_count:{}'.format(sentence_count))
    
    #sentence_count += 1
          
f1.close()

In [2]:
##### Bert Tokenizer 훈련 시작
from tokenizers import BertWordPieceTokenizer

print(f'BertWordPieceTokenizer=>')
WPT = BertWordPieceTokenizer(
    #vocab='/content/wordPieceTokenizer/wiki_20190620-vocab.txt', # 기존 vocab 넣어도 사전 학습 안됨
    clean_text=True,       # 토큰에서 공백은 제거 
    handle_chinese_chars=True,  # 한자는 모두 char 단위로 쪼갬
    strip_accents=False,  # True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
    lowercase=False       # 한국어에서는 반드시 False
)

print(f'train=>')
WPT.train(
    #files=["my_data/wiki_20190620_small.txt", "my_data/nsmc_subword.txt"],  #여러개 파일도 추가할수 있음
    files=[mecab_output],
    vocab_size=vocab_size,       # 만들고자 하는 vocab의 size, 보통 '32000' 정도가 좋다고 알려져 있다
    min_frequency=min_frequency,        # merge를 수행할 최소 빈도수, 2로 설정 시 2회 이상 등장한 pair만 수행한다
    show_progress=True,     # 학습 진행과정 show
    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],# Tokenizer에 추가하고 싶은 special token 지정
    wordpieces_prefix="##"
    #limit_alphabet=6000  # merge 수행 전 initial tokens이 유지되는 숫자 제한
    #initial_alphabet   # 꼭 포함됐으면 하는 initial alphabet, 이곳에 설정한 token은 학습되지 않고 그대로 포함되도록 설정된다.
)
print(f'save_model=>')
# vcoab_folder/vocab_ouput-vocab.txt 파일로 저장
# => 뒤에 vocab이 붙음
WPT.save_model(vocab_folder, vocab_output)

BertWordPieceTokenizer=>
train=>



save_model=>


['../../data11/ai_hub/vocab/tl1-1줄-mecab-30000-vocab.txt']

In [ ]:
### 이후 작업은 실제 BERT 토크너나이저 스페셜 토큰을 추가하여 만들때 사용하는 것이므로 pass해도 됨.**

In [3]:
print(WPT.get_vocab_size())

30000


In [4]:
text = "오늘은 경마장에서 조랑말 탄다"
tokenized_text = WPT.encode(text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

['오늘', '##은', '경마', '##장', '##에', '##서', '조', '##랑', '##말', '탄다']
[2477, 1261, 15785, 1207, 1039, 1107, 733, 1037, 1299, 27075]


In [5]:
##### speical 토큰 추가 및 json 파일 생성
from transformers import BertTokenizer, BertTokenizerFast, DistilBertTokenizer, DistilBertTokenizerFast
from transformers import BertTokenizerFast

In [6]:
vocab_path = vocab_folder + '/' + vocab_output + '-vocab.txt'
vocab_special_path =  vocab_folder + '/' + vocab_special_output

In [7]:
print(vocab_path)
print(vocab_special_path)

../../data11/ai_hub/vocab//tl1-1줄-mecab-30000-vocab.txt
../../data11/ai_hub/vocab//tl1-1줄-mecab-30000-speical


In [8]:
# BertTokenizerFast, BertTokenizer 둘중 하나를 사용하면 됨
'''
tokenizer = BertTokenizerFast(vocab_file=vocab_path, 
                              max_len=128, 
                              do_lower_case=False)
'''                              

#print(tokenizer.tokenize("뷁은 [MASK] 조선 중기의 무신이다."))
#tokenizer.add_special_tokens({'mask_token':'[MASK]'})

tokenizer = BertTokenizer(vocab_file=vocab_path, 
                          strip_accents=False, 
                          do_lower_case=False)

In [9]:
"""Special Token 지정

-special_tokens : 언어모델의 범용성을 위해 Dummy token, 여러 개의 [unused]와 [UNK]를 꼭 설정해야 한다.

-언어모델에 번역, 요약, 개체명 인식 모델을 fine-tuning시 Dummy token이 필요한 경우가 많다. 또 도메인 특화된 task를 수행할 땐 도메인 토큰을 따로 선언하는게 필수이다.

-이를 고려하지 않으면 편법을 사용해야 한다. ETRI의 KorBERT는 Dummy token이 없어 빈도수가 작은 token을 Dummy로 대체해서 쓰기도 한다.

-꼭 충분한 unused와 UNK를 설정하자. 그리고 BOS(문장 시작), EOS(문장 끝) 등등도 추가하자. 본 프로젝트에선 10개의 UNK와 200개의 unsed token을 선언하였다.



"""

special_tokens=['[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]',
                '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]',]


In [10]:
special_tokens_dict = {'additional_special_tokens': special_tokens}
tokenizer.add_special_tokens(special_tokens_dict)

22

In [11]:
# special token 체크
tokenizer.all_special_tokens

['[UNK]',
 '[SEP]',
 '[PAD]',
 '[CLS]',
 '[MASK]',
 '[BOS]',
 '[EOS]',
 '[UNK0]',
 '[UNK1]',
 '[UNK2]',
 '[UNK3]',
 '[UNK4]',
 '[UNK5]',
 '[UNK6]',
 '[UNK7]',
 '[UNK8]',
 '[UNK9]',
 '[unused0]',
 '[unused1]',
 '[unused2]',
 '[unused3]',
 '[unused4]',
 '[unused5]',
 '[unused6]',
 '[unused7]',
 '[unused8]',
 '[unused9]']

In [12]:
# special token 추가한 special vocab을 저장함
tokenizer.save_pretrained(vocab_special_path)

('../../data11/ai_hub/vocab//tl1-1줄-mecab-30000-speical/tokenizer_config.json',
 '../../data11/ai_hub/vocab//tl1-1줄-mecab-30000-speical/special_tokens_map.json',
 '../../data11/ai_hub/vocab//tl1-1줄-mecab-30000-speical/vocab.txt',
 '../../data11/ai_hub/vocab//tl1-1줄-mecab-30000-speical/added_tokens.json')

In [13]:
# special vocab 확인

tokenizer_check = BertTokenizerFast.from_pretrained(vocab_special_path)

print('check special tokens : %s'%tokenizer_check.all_special_tokens[:20])

print('vocab size : %d' % tokenizer_check.vocab_size)
tokenized_input_for_pytorch = tokenizer_check("나는 오늘 아침밥을 먹었다.", return_tensors="pt")
#tokenized_input_for_tensorflow = tokenizer_check("나는 오늘 아침밥을 먹었다.", return_tensors="tf")

print("Tokens (str)      : {}".format([tokenizer_check.convert_ids_to_tokens(s) for s in tokenized_input_for_pytorch['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(tokenized_input_for_pytorch['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(tokenized_input_for_pytorch['attention_mask'].tolist()[0]))


check special tokens : ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]']
vocab size : 30000
Tokens (str)      : ['[CLS]', '나', '##는', '오늘', '아침', '##밥', '##을', '먹', '##었', '##다', '.', '[SEP]']
Tokens (int)      : [2, 194, 1283, 2477, 3348, 1482, 1303, 411, 1282, 1044, 17, 3]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

